<a href="https://colab.research.google.com/github/saurabhIU/Deep-Learning/blob/master/Speech_Denoising_Using_2D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Preparation

In [24]:

import librosa
import librosa.display as disp
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time;
print(tf.__version__)

1.13.1


## Load Data (Training Clean and Training Noisy data)

In [25]:
s_clean, sr_clean=librosa.load('train_clean_male.wav',sr=None)
print(f'Shape of clean signal is {s_clean.shape}')
S_clean=librosa.stft(s_clean, n_fft=1024, hop_length=512)
S_clean_abs = np.abs(S_clean).T

s_dirty, sr_noisy=librosa.load('train_dirty_male.wav',sr=None)
print(f'Shape of clean signal is {s_dirty.shape}')
S_dirty=librosa.stft(s_dirty, n_fft=1024, hop_length=512)

S_dirty_abs = np.abs(S_dirty).T

Shape of clean signal is (1258899,)
Shape of clean signal is (1258899,)


In [26]:
print(f'Shape of clean signal is {S_clean_abs.shape} and shape of noisy signal is {S_dirty_abs.shape}')

Shape of clean signal is (2459, 513) and shape of noisy signal is (2459, 513)


## Prepare 2-D Image data out of spectrogram

In [37]:
# Create Training image data

image_height = 20 #Height of image
image_width = S_dirty_abs.shape[1]
number_of_images_to_train = S_dirty_abs.shape[0] - image_height +1
X_train = np.zeros((number_of_images_to_train,image_height,image_width))

# Create Training label data
number_of_train_label = S_clean_abs.shape[0] - image_height +1
Y_train = np.zeros((number_of_train_label,image_height,image_width))

print(f'Shape of train data is {X_train.shape} and shape of train label is {Y_train.shape}')


Shape of train data is (2440, 20, 513) and shape of train label is (2440, 20, 513)


## CNN Set-up

In [0]:
Convolution layer1: kernal size = [4,4], num of filters = 16, pool patch size = [2,2], pool strides size = [2,2] 
  
EPOCHS = 2000

kernel_size1 = 4
kernel_size2 = 2

conv_strides = [1,1,1,1]
max_pool_strides = [1,2,2,1]

Batch_Size = 200

kernel_num1 = 16
kernel_num2 = 32

fc1_size = 4000
fc1_size = 513

X = tf.placeholder("float", [None,20,513,1])
Y = tf.placeholder("float", [None,513])

In [0]:
filters = {
              'wl1': tf.get_variable('W1', shape=(kernel_size1,kernel_size1,1,kernel_num1), initializer=tf.initializers.he_normal()),
              'wl2': tf.get_variable('W2', shape=(kernel_size2,kernel_size2,kernel_num1,kernel_num2), initializer=tf.initializers.he_normal()),
              'wfc1': tf.get_variable('W4', shape=(4160,fc1_size), initializer=tf.initializers.he_normal()),
              'wfc2': tf.get_variable('W5', shape=(fc1_size,fc2_size), initializer=tf.initializers.he_normal()),
    
              
          }

biases = {
              'bl1': tf.get_variable('B1', shape=(kernel_num1), initializer=tf.initializers.he_normal()),
              'bl2': tf.get_variable('B2', shape=(kernel_num2), initializer=tf.initializers.he_normal()),
              'bl3': tf.get_variable('B4', shape=(fc1_size), initializer=tf.initializers.he_normal()),
              'bl4': tf.get_variable('B5', shape=(fc2_size), initializer=tf.initializers.he_normal()),
    
          }

In [0]:
#Convolution Layer 
# kernel tensor of shape [filter_width, in_channels, out_channels]

def build_convolutional_layer(input,filter_num,bias_num,kernel_num):
    
    
    conv_layer = tf.nn.conv2d(input,filters[filter_num],strides=conv_strides,padding='SAME')
    
    conv_layer += biases[bias_num]
    
    conv_layer = tf.nn.max_pool (value=conv_layer,
                                  ksize=max_pool_strides,
                                  strides=max_pool_strides,
                                  padding='SAME')
    
    conv_layer = tf.nn.relu(conv_layer)
    
    return conv_layer

In [0]:
def conv_nn(input):
  
  
  layer1 = build_convolutional_layer(input,'wl1','bl1',kernel_num1)
  print(layer1)
  
  layer2 = build_convolutional_layer(layer1,'wl2','bl2',kernel_num2)
  print(layer2)
  
  
  # Fully connected layer
  layer_flat, fc_feature_num  = flatten_layer(layer3)
  
  print(fc_feature_num)
  fc1 = tf.matmul(layer_flat,filters['wfc1']) + biases['bl3']
  
  fc1 = tf.nn.relu(fc1)
  
  fc2 = tf.matmul(fc1,filters['wfc2']) + biases['bl4']
  
  return fc2

In [0]:

logits = conv_nn(X)
cost = tf.losses.mean_squared_error(Y,logits)
optimizer = tf.train.AdamOptimizer().minimize(cost)